<font size=6>Coronavirus update with statistics and graphs</font>

From World Health Organization - On 31 December 2019, WHO was alerted to several cases of pneumonia in Wuhan City, Hubei Province of China. The virus is called <font color='red'>COVID-19</font>.
the number of affected people by this virus is in increase.

In this tutorial we will try to get acces to coronavirus data from <font size=3>https://datahub.io/core/covid-19/r/time-series-19-covid-combined.csv </font> and we will try to make it easy for as to know the evolution of the virus in each country in the wourld.


We will start by importing modules that we need

In [90]:
from datetime import datetime
from ipywidgets import interact_manual, interact,interactive
import ipywidgets as ipw
import pandas as pd
import plotly.graph_objects as graph

import data in a dataframe object

In [4]:
df=pd.read_csv('https://raw.githubusercontent.com/datasets/covid-19/master/data/time-series-19-covid-combined.csv')

we will display <font color='green'>5</font> lines to visualize data

In [5]:
df.head()

,Date,Country/Region,Province/State,Lat,Long,Confirmed,Recovered,Deaths
0,2020-01-22,Afghanistan,NaN,33.0,65.0,0.0,0.0,0.0
1,2020-01-23,Afghanistan,NaN,33.0,65.0,0.0,0.0,0.0
2,2020-01-24,Afghanistan,NaN,33.0,65.0,0.0,0.0,0.0
3,2020-01-25,Afghanistan,NaN,33.0,65.0,0.0,0.0,0.0
4,2020-01-26,Afghanistan,NaN,33.0,65.0,0.0,0.0,0.0


Our data countains date but in a string object , so will transform it to a datetime object in order to plot it thereafter

In [6]:
def convert_to_date(value):
    return datetime.strptime(value, '%Y-%m-%d')
df['Date'] = df['Date'].apply(convert_to_date)

then we will group the data by the country

In [92]:
def groupDataBycountry(country):
    gk=df.groupby('Country/Region')
    final_dataset=(gk.get_group(country))[['Date','Confirmed','Recovered','Deaths']]
    return final_dataset.set_index('Date')

extract from our grouped data the nombre of cases affected , number of deaths and the number of recovered cases of today

In [94]:
def extract_last_case(country):
    data=groupDataBycountry(country)
    d=data.iloc[-1]
    return (data.index[-1],d['Confirmed'],d['Recovered'],d['Deaths'])

we will define functions that plot the three cases

In [96]:
def plot_confirmed_cases(fig,data):
    fig.add_trace(graph.Scatter(
            x=data.index, y=data.Confirmed ,  name='confirmed',fill="tozeroy",
            marker_color='#77b5fe'
        ))
def plot_recovered_cases(fig,data):
    fig.add_trace(graph.Scatter(
            x=data.index, y=data.Recovered ,  name='recovered',fill="tozeroy",
            marker_color='rgb(63, 226, 101)'
        ))
def plot_deaths(fig,data):
    fig.add_trace(graph.Scatter(
            x=data.index, y=data.Deaths ,  name='deaths',fill="tozeroy",
            marker_color='rgb(255, 153, 153)'
        ))

the function bellow will apply the right theme to our graph (Light/Dark)

In [10]:
def apply_theme(fig,country,color_theme,font_color):
    fig.update_layout({
    'plot_bgcolor': color_theme,
    'paper_bgcolor': 'rgba(0,0,0,0)',
    })
    fig.update_layout(
        title={
        'text': "COVID-19 Evolution in "+country,
        'y':0.9,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'},
        xaxis_title="Date",
        yaxis_title="cases",
        legend=dict(
            x=0.01,
            y=1,
            traceorder="normal",
            font=dict(
                family="sans-serif",
                size=12,
                color=font_color
            ),
            bordercolor=color_theme,
            borderwidth=1
        ))
    fig.update_xaxes(showline=True, linewidth=1, linecolor='gray')
    fig.update_yaxes(showline=True, linewidth=1, linecolor='gray')

the function that plot the graph in function of the country chosen

In [110]:
def plot_graph(data,country,confirmed,recovered,deaths,theme):
    fig=graph.Figure()
    if(confirmed==True):
        plot_confirmed_cases(fig,data)
    if(recovered==True):
        plot_recovered_cases(fig,data)
    if(deaths==True):
        plot_deaths(fig,data)
    if(theme=="Dark"):
        color_theme="rgb(50,50,50)"
        font_color="white"
    else:
        color_theme="rgba(0,0,0,0)"
        font_color="black"
    apply_theme(fig,country,color_theme,font_color)
    fig.show()

then we will define the widgets used

In [111]:
dropdown=ipw.Dropdown(options=list(dict.fromkeys(df['Country/Region'].tolist())),description="country :")
deaths=ipw.Checkbox( 
    value=False,
    description='Deaths',
    disabled=False
)
recovered=ipw.Checkbox( 
    value=False,
    description='Recovered',
    disabled=False
)
confirmed=ipw.Checkbox( 
    value=True,
    description='Confirmed',
    disabled=False
)
theme=ipw.RadioButtons(
    options=['Light', 'Dark'],
    description="theme :",
    value='Light',
    disabled=False
)
v=extract_last_case(dropdown.value)
Title=ipw.HTML(description=" by BOUGRINE",value="<center><h3><font color='red'>COVID-19</font> Evolution in <font color='blue'>"+dropdown.value+"</font></h3></center>")
Title2=ipw.HTML(value="<h4><font >"+str(v[0])+" </font>  <font color='blue'>Confirmed : </font>"+str(v[1])+" <font color='green'>Recovered : </font>"+str(v[2])+" <font color='red'>Deaths : </font>"+str(v[3])+"</h4>")

execute is the function that call all the other functions written above

In [112]:
def execute(country,confirmed,recovered,deaths,theme):
    data=groupDataBycountry(country)
    v=extract_last_case(country)
    global Title
    global Title2
    Title=ipw.HTML(value="<center><h3><font color='red'>COVID-19</font> Evolution in <font color='blue'>"+country+"</font></h3></center>")
    Title2=ipw.HTML(value="<center><h4><font >"+str(v[0])+" </font>  <font color='blue'>Confirmed : </font>"+str(v[1])+" <font color='green'>Recovered : </font>"+str(v[2])+" <font color='red'>Deaths : </font>"+str(v[3])+"</h4></center>")
    display(Title,Title2)
    return plot_graph(data,country,confirmed,recovered,deaths,theme)

and finally we write the function wich will be called whenever we interact with the widgets

In [122]:
display(ipw.HTML(value="<center><h2>Choose <font color='REd'>the country</font> ,<font color='Blue'>the cases</font> you want to see and <font color='Green'>the theme</font> mode</h2></center>"))
def final(a,b,c,d,e):
    execute(dropdown.value,confirmed.value,recovered.value,deaths.value,theme.value)
w=interactive(final,a=dropdown,b=deaths,c=recovered,d=confirmed,e=theme)
display(w)

HTML(value="<center><h2>Choose <font color='REd'>the country</font> ,<font color='Blue'>the cases</font> you w…

interactive(children=(Dropdown(description='country :', options=('Afghanistan', 'Albania', 'Algeria', 'Andorra…